In [2]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

# --- 1. Data Loading, Cleaning, and Preparation ---

# Read data from the file and handle common null values
df = pd.read_csv('orders.csv', na_values=['Not Available', 'unknown'])

# Rename columns to be lower case and replace spaces with underscores
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')

# Derive new columns: discount, sale_price, and profit
# Assuming 'list_price', 'discount_percent', and 'cost_price' columns exist
df['discount'] = df['list_price'] * df['discount_percent'] * 0.01
df['sale_price'] = df['list_price'] - df['discount']
df['profit'] = df['sale_price'] - df['cost_price']

# Define the columns and their order for insertion (must match SQL)
columns_to_insert = [
    'order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city', 
    'state', 'postal_code', 'region', 'category', 'sub_category', 
    'product_id', 'quantity', 'discount', 'sale_price', 'profit'
]

# Create a clean DataFrame for insertion
df_insert = df[columns_to_insert].copy() 

# *** CRITICAL FIX for the 'Unknown column 'nan'' error ***
# Replace all pandas NaN (Not a Number) values with Python's None.
# The mysql.connector correctly translates 'None' to 'NULL' in the database.
df_insert = df_insert.where(pd.notnull(df_insert), None) 


# --- 2. MySQL Connection and Insertion ---

connection = None
try:
    # Step 1: Connect to MySQL
    connection = mysql.connector.connect(
        host='localhost',          # or your MySQL server IP
        user='root',               # your MySQL username
        password='Sarran@88',      # your MySQL password
        database='sales_discount'  # your database name
    )

    if connection.is_connected():
        print("✅ Connected to MySQL successfully!")
        cursor = connection.cursor()

        # Step 2: Create table if not exists
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS orders_table (
            order_id INT,
            order_date DATE,
            ship_mode VARCHAR(50),
            segment VARCHAR(100),
            country VARCHAR(50),
            city VARCHAR(50),
            state VARCHAR(50),
            postal_code INT,
            region VARCHAR(50),
            category VARCHAR(50),
            sub_category VARCHAR(100),
            product_id VARCHAR(100),
            quantity INT,
            discount FLOAT,
            sale_price FLOAT,
            profit FLOAT
        )
        """)
        
        print("📝 Table 'orders_table' ready.")


        # Step 3: Define the SQL INSERT statement
        sql = """
        INSERT INTO orders_table (order_id, order_date, ship_mode, segment, country, city, state, postal_code,
        region, category, sub_category, product_id, quantity, discount, sale_price, profit) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        
        # Step 4: Iterate through the DataFrame and insert data
        print("⏳ Starting data insertion...")
        
        rows_inserted = 0
        for i, row in df_insert.iterrows():
            # Convert the row to a tuple of Python standard types (not NumPy)
            data_to_insert = tuple(row.to_list())
            
            # Execute the query for the current row
            cursor.execute(sql, data_to_insert)
            rows_inserted += 1

        # Step 5: Commit the changes
        connection.commit()
        print(f"🎉 Data insertion complete! Successfully inserted **{rows_inserted}** rows.")


except Error as e:
    print("❌ Error during MySQL operation:", e)

finally:
    # Step 6: Close the connection
    if connection and connection.is_connected():
        cursor.close()
        connection.close()
        print("🔌 MySQL connection closed.")

✅ Connected to MySQL successfully!
📝 Table 'orders_table' ready.
⏳ Starting data insertion...
🎉 Data insertion complete! Successfully inserted **9994** rows.
🔌 MySQL connection closed.
